In [1]:
%matplotlib inline
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
#import statsmodels.api as sm
from datetime import datetime

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

In [2]:
customers = pd.read_csv("customers5.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

# CUSTOMERS

In [3]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,17860216.0,GB,22/10/2015 15:00,Femme,17/10/2015 12:58
1,10588204.0,FR,20/10/2015 19:46,Femme,20/10/2015 00:07
2,17841489.0,DE,20/10/2015 16:40,Homme,12/10/2015 23:04
3,17849688.0,DE,15/10/2015 14:17,Homme,22/10/2015 10:41
4,12916644.0,FR,13/10/2015 15:54,Femme,15/03/2012 10:04


### BirthDate -> Age

In [5]:
customers['BirthDate'] = customers['BirthDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['Age'] = 2018 - customers['BirthDate'].apply(lambda x : x.year)

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age
0,17860216.0,GB,2015-10-22 15:00:00,Femme,17/10/2015 12:58,3
1,10588204.0,FR,2015-10-20 19:46:00,Femme,20/10/2015 00:07,3
2,17841489.0,DE,2015-10-20 16:40:00,Homme,12/10/2015 23:04,3
3,17849688.0,DE,2015-10-15 14:17:00,Homme,22/10/2015 10:41,3
4,12916644.0,FR,2015-10-13 15:54:00,Femme,15/03/2012 10:04,3


In [17]:
for i in range(10,60,5):
    customers['Age' + str(i) + '-' + str(i+5)] = ((customers['Age']>i) & (customers['Age']<=i+5)).astype('float')

In [23]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,Age50-55,Age55-60
0,17860216.0,GB,2015-10-22 15:00:00,Femme,17/10/2015 12:58,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,20/10/2015 00:07,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,12/10/2015 23:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,22/10/2015 10:41,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,15/03/2012 10:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Gender -> Binaire